## Part 1: Preprocessing

In [47]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [48]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [49]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [50]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
                    'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
                    'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes



Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [51]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [52]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
label_encoder = LabelEncoder()
X_train['OverTime'] = label_encoder.fit_transform(X_train['OverTime'])
X_test['OverTime'] = label_encoder.transform(X_test['OverTime'])
print(attrition_df['OverTime'].value_counts())

No     1054
Yes     416
Name: OverTime, dtype: int64


In [53]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [54]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = department_encoder.transform(y_train[['Department']])
y_test_department_encoded = department_encoder.transform(y_test[['Department']])

print(y_train_department_encoded)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [55]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

print(y_train_attrition_encoded)


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Create, Compile, and Train the Model

In [56]:
# Find the number of columns in the X training data
input_shape = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_shape,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)



In [57]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)

In [58]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
# Assuming there are 2 possible outputs for Attrition based on the one-hot encoded shape
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [59]:
# correcting shared layer names based on expected output
shared_layer_1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer_2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer_1)

# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        "department_output": "categorical_crossentropy",
        "attrition_output": "binary_crossentropy"
    },
    metrics=["accuracy"]
)


# Summarize the model
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 64)                   704       ['input_3[0][0]']             
                                                                                                  
 dense_11 (Dense)            (None, 32)                   2080      ['dense_10[0][0]']            
                                                                                                  
 dense_12 (Dense)            (None, 16)                   528       ['dense_11[0][0]']            
                                                                                            

In [60]:
# Train the model
history = model.fit(
    x=X_train_scaled,
    y={
        "department_output": y_train_department_encoded,
        "attrition_output": y_train['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0).values
    },
    validation_data=(
        X_test_scaled,
        {
            "department_output": y_test_department_encoded,
            "attrition_output": y_test['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0).values
        }
    ),
    epochs=100,
    batch_size=32
)




Epoch 1/100
37/37 [==============================] - 2s 14ms/step - loss: 1.5241 - department_output_loss: 0.8606 - attrition_output_loss: 0.6635 - department_output_accuracy: 0.6182 - attrition_output_accuracy: 0.6131 - val_loss: 1.3422 - val_department_output_loss: 0.7925 - val_attrition_output_loss: 0.5497 - val_department_output_accuracy: 0.6667 - val_attrition_output_accuracy: 0.8673
Epoch 2/100
37/37 [==============================] - 0s 5ms/step - loss: 1.2808 - department_output_loss: 0.7788 - attrition_output_loss: 0.5020 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8308 - val_loss: 1.1905 - val_department_output_loss: 0.7838 - val_attrition_output_loss: 0.4067 - val_department_output_accuracy: 0.6667 - val_attrition_output_accuracy: 0.8673
Epoch 3/100
37/37 [==============================] - 0s 6ms/step - loss: 1.2038 - department_output_loss: 0.7676 - attrition_output_loss: 0.4362 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8316

In [62]:
# Evaluate the model with the testing data
evaluation_results = model.evaluate(X_test_scaled,
                                    {"department_output": y_test_department_encoded,
                                     "attrition_output": y_test['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0).values})
print(evaluation_results)

10/10 [==============================] - 0s 9ms/step - loss: 2.2853 - department_output_loss: 1.5876 - attrition_output_loss: 0.6977 - department_output_accuracy: 0.5204 - attrition_output_accuracy: 0.8231
[2.2852730751037598, 1.5876046419143677, 0.6976681351661682, 0.5204081535339355, 0.8231292366981506]


In [63]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {evaluation_results[3]}")
print(f"Attrition predictions accuracy: {evaluation_results[4]}")

Department predictions accuracy: 0.5204081535339355
Attrition predictions accuracy: 0.8231292366981506


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric, particularly for imbalanced datasets. Precision, recall, or F1 scores could provide more insights for imbalanced classes.

2. For the department prediction, softmax was used because it's great for multi-class classification, giving us probabilities that sum up to 1. For attrition, a binary outcome, sigmoid was chosen since it outputs a probability between 0 and 1, perfect for yes/no (binary) decisions.

3. Initial improvements could focus on fine-tuning hyperparameters to optimize model performance as well as incorporating dropout techniques to mitigate overfitting.